In [1]:
import numpy as np
import os
from  natsort import natsorted
import imageio
import re
import time
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, GRU, Reshape
from keras.layers import Conv2D, MaxPooling2D, Bidirectional, Permute, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.models import load_model, Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import itertools

Using TensorFlow backend.


In [2]:
NAME = 'ALPR_CNN'
data_dir = 'Data'
model_dir = 'Models'

In [5]:
def prepare_dataset(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load(os.path.join(data_dir,'X_{}.npy'.format(folder_name)))
        y = np.load(os.path.join(data_dir,'y_{}.npy'.format(folder_name)))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[0][2:])
            print(label[0][2:])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y

In [6]:
X,y = prepare_dataset(data_dir, 'resized')

Loading numpy


In [7]:
X.shape

(3921, 100, 160, 3)

In [8]:
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [9]:
z = 3500
X_train = X[:z]
X_test = X[z:]
y_train = y[:z]
y_test = y[z:]

In [10]:
regex = r'^[A-Z0-9 ]+$'
alphabet = u'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '

In [11]:
# Translation of characters to unique integer values
def text_to_labels(text):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


# Reverse translation of numerical classes back to characters
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)


# only a-z and space..probably not to difficult
# to expand to uppercase and symbols

def is_valid_str(in_str):
    search = re.compile(regex, re.UNICODE).search
    return bool(search(in_str))

In [12]:
#Removing 7 label plates
mylen = np.vectorize(len)

count = mylen(y_train)
X_train = np.delete(X_train, np.where(count>6)[0], axis=0 )
y_train = np.delete(y_train, np.where(count>6)[0], axis=0 )

count = mylen(y_test)
X_test = np.delete(X_test, np.where(count>6)[0], axis=0 )
y_test = np.delete(y_test, np.where(count>6)[0], axis=0 )

In [13]:
def code_labels(labels):
    y_coded = []
    for l in labels:
        y_coded.append(text_to_labels(l))
    return np.array(y_coded)

In [14]:
y_train_coded = code_labels(y_train)
y_test_coded = code_labels(y_test)

In [15]:
num_classes = 36
#Labels to binary
y_train_0 = keras.utils.to_categorical(y_train_coded[:,0],num_classes)
y_train_1 = keras.utils.to_categorical(y_train_coded[:,1],num_classes)
y_train_2 = keras.utils.to_categorical(y_train_coded[:,2],num_classes)
y_train_3 = keras.utils.to_categorical(y_train_coded[:,3],num_classes)
y_train_4 = keras.utils.to_categorical(y_train_coded[:,4],num_classes)
y_train_5 = keras.utils.to_categorical(y_train_coded[:,5],num_classes)

y_test_0 = keras.utils.to_categorical(y_test_coded[:,0],num_classes)
y_test_1 = keras.utils.to_categorical(y_test_coded[:,1],num_classes)
y_test_2 = keras.utils.to_categorical(y_test_coded[:,2],num_classes)
y_test_3 = keras.utils.to_categorical(y_test_coded[:,3],num_classes)
y_test_4 = keras.utils.to_categorical(y_test_coded[:,4],num_classes)
y_test_5 = keras.utils.to_categorical(y_test_coded[:,5],num_classes)


In [110]:
from collections import Counter
Counter(count)

Counter({6: 3495, 7: 5})

## Keras Model

In [16]:
img_w = 100
# Input Parameters
img_h = 160
# Network parameters
conv_filters = 64
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 64
minibatch_size = 128
unique_tokens = 37

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_w, img_h)
else:
    input_shape = (img_w, img_h, 3)


In [17]:
num_feat_map = 64
p = 0.2

In [18]:
input_data = Input(name='the_input', shape=input_shape, dtype='float32')
x = Conv2D(num_feat_map, kernel_size=(3, 3),
               activation='relu',
               input_shape=input_shape,
               padding='same', name='Conv_1')(input_data)
x = BatchNormalization(name='Bn_1')(x)
x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
x = Dropout(p, name='Drop_1')(x)
x = Conv2D(num_feat_map, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
x = BatchNormalization(name='Bn_2')(x)
x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_2')(x)
x = Dropout(p, name='Drop_2')(x)
# model.add(Conv2D(num_feat_map, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3'))
# model.add(BatchNormalization(name='Bn_3'))
# model.add(MaxPooling2D(pool_size=(2, 2), name='Max_pool_3'))
# model.add(Dropout(p, name='Drop_3'))
x = Permute((2, 1, 3), name='Permute_1')(x)  # for swap-dimension
x = Reshape((-1, num_feat_map * (img_w // (pool_size ** 2))), name='Reshape_1')(x)
x = Dense(time_dense_size, activation='relu', name='dense1')(x)
x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
x = BatchNormalization(name='Bn_4')(x)
x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
x = BatchNormalization(name='Bn_5')(x)
x = Dropout(p, name='Drop_4')(x)
#x = Dense(128, activation='relu', name='dense_2')(x)
x = Flatten()(x)
x = Dense(128, activation='relu', name='dense_3')(x)
x = BatchNormalization(name='Bn_6')(x)
x = Dropout(p, name='Drop_5')(x)

#output
d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(x)
d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(x)
d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(x)
d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(x)
d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(x)
d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(x)
model = Model(input_data, [d1,d2,d3,d4,d5,d6])


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 100, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 100, 160, 64) 1792        the_input[0][0]                  
__________________________________________________________________________________________________
Bn_1 (BatchNormalization)       (None, 100, 160, 64) 256         Conv_1[0][0]                     
__________________________________________________________________________________________________
Max_pool_1 (MaxPooling2D)       (None, 50, 80, 64)   0           Bn_1[0][0]                       
__________________________________________________________________________________________________
Drop_1 (Dr

In [19]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

batch_size = 50
epochs = 100


In [20]:
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
t = int(time.time())

# checkpoint
chk_path = os.path.join(model_dir, 'best_{}_{}'.format(NAME,t))
checkpoint = ModelCheckpoint(chk_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir="logs/{}_{}".format(NAME,t))
callbacks_list = [checkpoint, tensorboard]

history = model.fit(X_train, [y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5],
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            shuffle=True,
            validation_data=(X_test, [y_test_0, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]),
            callbacks=callbacks_list)

#Saving the model
model.save(os.path.join(model_dir, 'final_{}_{}'.format(NAME,t)))

Train on 3495 samples, validate on 420 samples
Epoch 1/100
3495/3495 [==============================] - 33s 10ms/step - loss: 21.9424 - dense_out_1_loss: 3.8772 - dense_out_2_loss: 3.7652 - dense_out_3_loss: 3.6169 - dense_out_4_loss: 3.5399 - dense_out_5_loss: 3.5453 - dense_out_6_loss: 3.5979 - dense_out_1_acc: 0.0821 - dense_out_2_acc: 0.1090 - dense_out_3_acc: 0.1379 - dense_out_4_acc: 0.1662 - dense_out_5_acc: 0.1651 - dense_out_6_acc: 0.1585 - val_loss: 18.9082 - val_dense_out_1_loss: 3.4173 - val_dense_out_2_loss: 3.3931 - val_dense_out_3_loss: 3.1071 - val_dense_out_4_loss: 2.8294 - val_dense_out_5_loss: 3.0328 - val_dense_out_6_loss: 3.1286 - val_dense_out_1_acc: 0.1857 - val_dense_out_2_acc: 0.1690 - val_dense_out_3_acc: 0.2571 - val_dense_out_4_acc: 0.2929 - val_dense_out_5_acc: 0.3119 - val_dense_out_6_acc: 0.3119


c:\programdata\anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:432: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/100
3495/3495 [==============================] - 26s 7ms/step - loss: 14.6540 - dense_out_1_loss: 2.7898 - dense_out_2_loss: 2.7984 - dense_out_3_loss: 2.4049 - dense_out_4_loss: 2.0668 - dense_out_5_loss: 2.1974 - dense_out_6_loss: 2.3967 - dense_out_1_acc: 0.3199 - dense_out_2_acc: 0.3050 - dense_out_3_acc: 0.3957 - dense_out_4_acc: 0.4701 - dense_out_5_acc: 0.4592 - dense_out_6_acc: 0.4206 - val_loss: 12.4871 - val_dense_out_1_loss: 2.2851 - val_dense_out_2_loss: 2.5402 - val_dense_out_3_loss: 2.0613 - val_dense_out_4_loss: 1.6777 - val_dense_out_5_loss: 1.7412 - val_dense_out_6_loss: 2.1816 - val_dense_out_1_acc: 0.4333 - val_dense_out_2_acc: 0.3810 - val_dense_out_3_acc: 0.4571 - val_dense_out_4_acc: 0.5619 - val_dense_out_5_acc: 0.5381 - val_dense_out_6_acc: 0.4595
Epoch 3/100
3495/3495 [==============================] - 25s 7ms/step - loss: 9.9673 - dense_out_1_loss: 1.9058 - dense_out_2_loss: 2.0905 - dense_out_3_loss: 1.6455 - dense_out_4_loss: 1.2085 - dense_out_5_los

3495/3495 [==============================] - 25s 7ms/step - loss: 1.5501 - dense_out_1_loss: 0.3234 - dense_out_2_loss: 0.3076 - dense_out_3_loss: 0.2607 - dense_out_4_loss: 0.1804 - dense_out_5_loss: 0.2099 - dense_out_6_loss: 0.2682 - dense_out_1_acc: 0.9193 - dense_out_2_acc: 0.9236 - dense_out_3_acc: 0.9391 - dense_out_4_acc: 0.9536 - dense_out_5_acc: 0.9488 - dense_out_6_acc: 0.9345 - val_loss: 3.3127 - val_dense_out_1_loss: 0.5717 - val_dense_out_2_loss: 0.6290 - val_dense_out_3_loss: 0.6364 - val_dense_out_4_loss: 0.4484 - val_dense_out_5_loss: 0.4023 - val_dense_out_6_loss: 0.6249 - val_dense_out_1_acc: 0.8571 - val_dense_out_2_acc: 0.8500 - val_dense_out_3_acc: 0.8357 - val_dense_out_4_acc: 0.8976 - val_dense_out_5_acc: 0.8833 - val_dense_out_6_acc: 0.8429
Epoch 13/100
3495/3495 [==============================] - 25s 7ms/step - loss: 1.3384 - dense_out_1_loss: 0.2723 - dense_out_2_loss: 0.2687 - dense_out_3_loss: 0.2309 - dense_out_4_loss: 0.1576 - dense_out_5_loss: 0.1827 - d

3495/3495 [==============================] - 26s 7ms/step - loss: 0.4828 - dense_out_1_loss: 0.1018 - dense_out_2_loss: 0.0936 - dense_out_3_loss: 0.0832 - dense_out_4_loss: 0.0575 - dense_out_5_loss: 0.0661 - dense_out_6_loss: 0.0808 - dense_out_1_acc: 0.9768 - dense_out_2_acc: 0.9783 - dense_out_3_acc: 0.9817 - dense_out_4_acc: 0.9871 - dense_out_5_acc: 0.9857 - dense_out_6_acc: 0.9800 - val_loss: 2.8966 - val_dense_out_1_loss: 0.4848 - val_dense_out_2_loss: 0.5213 - val_dense_out_3_loss: 0.5258 - val_dense_out_4_loss: 0.3956 - val_dense_out_5_loss: 0.3770 - val_dense_out_6_loss: 0.5922 - val_dense_out_1_acc: 0.8810 - val_dense_out_2_acc: 0.8667 - val_dense_out_3_acc: 0.8690 - val_dense_out_4_acc: 0.9143 - val_dense_out_5_acc: 0.9048 - val_dense_out_6_acc: 0.8690
Epoch 23/100
3495/3495 [==============================] - 25s 7ms/step - loss: 0.4437 - dense_out_1_loss: 0.0930 - dense_out_2_loss: 0.0853 - dense_out_3_loss: 0.0755 - dense_out_4_loss: 0.0493 - dense_out_5_loss: 0.0609 - d

...

3495/3495 [==============================] - 25s 7ms/step - loss: 0.1918 - dense_out_1_loss: 0.0386 - dense_out_2_loss: 0.0374 - dense_out_3_loss: 0.0267 - dense_out_4_loss: 0.0231 - dense_out_5_loss: 0.0283 - dense_out_6_loss: 0.0378 - dense_out_1_acc: 0.9900 - dense_out_2_acc: 0.9903 - dense_out_3_acc: 0.9928 - dense_out_4_acc: 0.9931 - dense_out_5_acc: 0.9908 - dense_out_6_acc: 0.9903 - val_loss: 3.0629 - val_dense_out_1_loss: 0.4252 - val_dense_out_2_loss: 0.5244 - val_dense_out_3_loss: 0.5735 - val_dense_out_4_loss: 0.4135 - val_dense_out_5_loss: 0.3860 - val_dense_out_6_loss: 0.7403 - val_dense_out_1_acc: 0.8976 - val_dense_out_2_acc: 0.8929 - val_dense_out_3_acc: 0.8881 - val_dense_out_4_acc: 0.9190 - val_dense_out_5_acc: 0.9262 - val_dense_out_6_acc: 0.8786
Epoch 53/100
3495/3495 [==============================] - 25s 7ms/step - loss: 0.1990 - dense_out_1_loss: 0.0447 - dense_out_2_loss: 0.0403 - dense_out_3_loss: 0.0277 - dense_out_4_loss: 0.0215 - dense_out_5_loss: 0.0336 - d

3495/3495 [==============================] - 25s 7ms/step - loss: 0.1748 - dense_out_1_loss: 0.0294 - dense_out_2_loss: 0.0371 - dense_out_3_loss: 0.0323 - dense_out_4_loss: 0.0215 - dense_out_5_loss: 0.0251 - dense_out_6_loss: 0.0293 - dense_out_1_acc: 0.9926 - dense_out_2_acc: 0.9920 - dense_out_3_acc: 0.9894 - dense_out_4_acc: 0.9940 - dense_out_5_acc: 0.9926 - dense_out_6_acc: 0.9917 - val_loss: 3.1399 - val_dense_out_1_loss: 0.5394 - val_dense_out_2_loss: 0.4692 - val_dense_out_3_loss: 0.5227 - val_dense_out_4_loss: 0.4662 - val_dense_out_5_loss: 0.3733 - val_dense_out_6_loss: 0.7692 - val_dense_out_1_acc: 0.8976 - val_dense_out_2_acc: 0.8976 - val_dense_out_3_acc: 0.8905 - val_dense_out_4_acc: 0.9190 - val_dense_out_5_acc: 0.9143 - val_dense_out_6_acc: 0.8738
Epoch 63/100
3495/3495 [==============================] - 26s 7ms/step - loss: 0.1693 - dense_out_1_loss: 0.0324 - dense_out_2_loss: 0.0288 - dense_out_3_loss: 0.0330 - dense_out_4_loss: 0.0188 - dense_out_5_loss: 0.0245 - d

3495/3495 [==============================] - 25s 7ms/step - loss: 0.1584 - dense_out_1_loss: 0.0303 - dense_out_2_loss: 0.0328 - dense_out_3_loss: 0.0217 - dense_out_4_loss: 0.0220 - dense_out_5_loss: 0.0208 - dense_out_6_loss: 0.0307 - dense_out_1_acc: 0.9911 - dense_out_2_acc: 0.9911 - dense_out_3_acc: 0.9954 - dense_out_4_acc: 0.9937 - dense_out_5_acc: 0.9940 - dense_out_6_acc: 0.9903 - val_loss: 3.0344 - val_dense_out_1_loss: 0.5259 - val_dense_out_2_loss: 0.4709 - val_dense_out_3_loss: 0.5530 - val_dense_out_4_loss: 0.4922 - val_dense_out_5_loss: 0.3228 - val_dense_out_6_loss: 0.6697 - val_dense_out_1_acc: 0.8905 - val_dense_out_2_acc: 0.9000 - val_dense_out_3_acc: 0.8952 - val_dense_out_4_acc: 0.9262 - val_dense_out_5_acc: 0.9238 - val_dense_out_6_acc: 0.8738
Epoch 73/100
3495/3495 [==============================] - 25s 7ms/step - loss: 0.1386 - dense_out_1_loss: 0.0263 - dense_out_2_loss: 0.0282 - dense_out_3_loss: 0.0242 - dense_out_4_loss: 0.0143 - dense_out_5_loss: 0.0205 - d

3495/3495 [==============================] - 25s 7ms/step - loss: 0.1580 - dense_out_1_loss: 0.0353 - dense_out_2_loss: 0.0304 - dense_out_3_loss: 0.0266 - dense_out_4_loss: 0.0169 - dense_out_5_loss: 0.0211 - dense_out_6_loss: 0.0276 - dense_out_1_acc: 0.9908 - dense_out_2_acc: 0.9894 - dense_out_3_acc: 0.9908 - dense_out_4_acc: 0.9951 - dense_out_5_acc: 0.9934 - dense_out_6_acc: 0.9920 - val_loss: 3.1957 - val_dense_out_1_loss: 0.5899 - val_dense_out_2_loss: 0.4691 - val_dense_out_3_loss: 0.6197 - val_dense_out_4_loss: 0.4596 - val_dense_out_5_loss: 0.3473 - val_dense_out_6_loss: 0.7102 - val_dense_out_1_acc: 0.8881 - val_dense_out_2_acc: 0.9071 - val_dense_out_3_acc: 0.8905 - val_dense_out_4_acc: 0.9333 - val_dense_out_5_acc: 0.9357 - val_dense_out_6_acc: 0.8786
Epoch 83/100
3495/3495 [==============================] - 25s 7ms/step - loss: 0.1439 - dense_out_1_loss: 0.0312 - dense_out_2_loss: 0.0305 - dense_out_3_loss: 0.0195 - dense_out_4_loss: 0.0166 - dense_out_5_loss: 0.0221 - d

3495/3495 [==============================] - 25s 7ms/step - loss: 0.1386 - dense_out_1_loss: 0.0216 - dense_out_2_loss: 0.0282 - dense_out_3_loss: 0.0288 - dense_out_4_loss: 0.0200 - dense_out_5_loss: 0.0163 - dense_out_6_loss: 0.0235 - dense_out_1_acc: 0.9951 - dense_out_2_acc: 0.9917 - dense_out_3_acc: 0.9940 - dense_out_4_acc: 0.9926 - dense_out_5_acc: 0.9957 - dense_out_6_acc: 0.9923 - val_loss: 3.5927 - val_dense_out_1_loss: 0.5228 - val_dense_out_2_loss: 0.5495 - val_dense_out_3_loss: 0.7053 - val_dense_out_4_loss: 0.5945 - val_dense_out_5_loss: 0.4092 - val_dense_out_6_loss: 0.8115 - val_dense_out_1_acc: 0.9000 - val_dense_out_2_acc: 0.9000 - val_dense_out_3_acc: 0.8738 - val_dense_out_4_acc: 0.9143 - val_dense_out_5_acc: 0.9238 - val_dense_out_6_acc: 0.8690
Epoch 93/100
3495/3495 [==============================] - 25s 7ms/step - loss: 0.1804 - dense_out_1_loss: 0.0338 - dense_out_2_loss: 0.0303 - dense_out_3_loss: 0.0339 - dense_out_4_loss: 0.0157 - dense_out_5_loss: 0.0299 - d